# Steps to run the deep learning epigenetic clock

Author: Jose Jaime Martinez Magana

Day: 20 February 2023

This script will run different epigenetic clocks in R
Follow this github https://github.com/yiluyucheng/dnaMethyAge for a complete description of the epigenetic clocks.

To run this clocks, the beta matrices should be normalized by BMIQ.

In [ ]:
# if you are using slurm job handler you could request resources with the following command
# uncomment the next line for job request
srun --pty --mem=32G --ntasks=4 -p interactive bash
# if your server uses modules to handle packages and conda/anaconda/miniconda is install load your conda
# uncomment the next line if you have miniconda installed in your server
module load miniconda
# conda activate environmnet
conda activate epigenetic_clocks
# this step should be included in a bash script

In [ ]:
# content of the epigenetic_clock.Rscript

#!/usr/bin/env Rscript --vanilla --slave
##################################################################################################
# R script for multiple epigenetic clocks
# day: 10 February 2023
# author: Jose Jaime Martinez-Magana
####################################################################################

# This script uses three inputs the qcdata following this github https://github.com/martinezjaime/ewas_saliva_ses/blob/main/qc_data/probe_and_sample_quality_control.ipynb,
# a list of samples to be included in the analysis as csv file

####################################################################################
# set parameters
# this script uses the library optparse to add arguments to the script
# adding arguments to the script
library(optparse) 
option_list=list(
    make_option(c("--file"),
                type="character",
                default=NULL,
                help="path to *rds object with qc information", metavar="character"),
    make_option(c("--out"),
                type="character",
                default="out.rds",
                help="output file name for rds [default= %default]", metavar="character"),
    make_option(c("--samplelist"),
                type="character",
                default=NULL,
                help="path to a *csv file with a list of samples to subset the analysis. This script uses a file with SampleID with Array_Sentrix structure"),
    make_option(c("--covar"),
                type="character",
                default=NULL,
                help="add the columns names for age and sex from the phenofile separated by ','", metavar="character")
    make_option(c("--outname"),
                type="character",
                default=NULL,
                help="name of the rds file after performing this analysis", metavar="character")
);

opt_parser=OptionParser(option_list=option_list);
opt=parse_args(opt_parser);

if (is.null(opt$file)){
  print_help(opt_parser)
  stop("At least one argument must be supplied (input file)", call.=FALSE)
}

In [ ]:
####################################################################################
# testing script
# this section will be removed from the final assoc_linear_450K.Rscript
opt=list()
opt$file="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/databases/qced/qced_data_v02062023.rds"
opt$out="/vast/palmer/scratch/montalvo-ortiz/jjm262/epigenomics/ewas_saliva_ses/results/epigenetic_age/"
opt$outname="epigenetic_clocks_all_v02102023.rds"
opt$covar=c("age","gender")
######################################################################################

In [ ]:
# loading library
library(dnaMethyAge)
library(BiocManager)
library(stringi)
library(parallel)
library(preprocessCore)

# developing script
# loading rds
paste0("Start analysis of data:",Sys.time(),"---","###Analysis path[",opt$file,"]###")
file=readRDS(opt$file)
# setting output file
outfile=paste0(opt$out,opt$outname,sep="")
paste0("Output of epigenetic clocks will be saved to:",Sys.time(),"---","###Output path[",outfile,"]###")

# loading phenotype file
paste0("Loading pheno file:",Sys.time())
# add covariates to be tested
covar=unlist(strsplit(opt$covar,','))
# get covars from pheno file
covars=c(covar)
covars=c(covar,"SampleID")
# select covars from phenofile
pheno_data=file$pheno[,colnames(file$pheno) %in% covars]
paste0('Warning, this script expects that sex is codify as 0 for females and 1 for males')
paste0('If you do not codify like this please do it')
# add SampleID as rownames
rownames(pheno_data)=pheno_data$SampleID
# add sample column to data frame
pheno_data$Sample=pheno_data$SampleID
# remove SampleID column
pheno_data$SampleID=NULL
# add age column
pheno_data$Age=pheno_data[covar[1]][,1]
# add age column
pheno_data$Sex=pheno_data[covar[2]][,1]
# recoding sex
pheno_data$Sex=ifelse(pheno_data$Sex == 0, "Female", "Male")


# get cells
paste0("Adding cells to pheno file:",Sys.time())
cells=file$cell_epidish
# transform cells to data frame
cells2=as.data.frame(cells)
# remove unneccesary object
rm(cells)
# removing cells with cero numbers in all samples
cells3=cells2[,colSums(cells2) > 0.00]
paste0("Warning: This script merges CD4T + CD8T cells to Limphocytes imputed with epiDISH from saliva, please revised the cell distribution in your data")
# merging CD4T and CD8T cells
cells3_m=cells3[,"CD4T", drop = F] + cells3[,"CD8T", drop = F]
colnames(cells3_m)="Lympho"
cells3_m2=cbind(cells3_m, cells3[,c("Epi","Fib","B","NK","Mono","Neutro")])
# add warning to say the user the cells that will be used in the analysis
paste0("Using the following cells in the regression models:",colnames(cells3_m2))
# merge pheno_data with cells
pheno_data=cbind(pheno_data,cells3_m2)

# subset samples if sample list is added
# subset phenofile
# here we used is.null, to ask if there was a sample list added to the script
if(!is.null(opt$samplelist)){
    paste0("Sample filter added, phenofile will be filtered based in:",opt$samplelist,"---",Sys.time())
    sample_f=read.csv(opt$samplelist)
    # subsetting pheno file
    pheno_data=pheno_data[rownames(pheno_data) %in% sample_f$SampleID,]
} else {
    pheno_data=pheno_data
}

# subset beta matrix
if(!is.null(opt$samplelist)){
    paste0("Sample filter added, beta matrix will be filtered based in:",opt$samplelist,"---",Sys.time())
    sample_f=read.csv(opt$samplelist)
    bvalues=file$rgsetraw_bmiq
    bvalues=bvalues[,colnames(bvalues) %in% sample_f$SampleID]
} else {
    bvalues=file$rgsetraw_bmiq
}

# sample sanity check
all_samples=is.null(pheno_data[!rownames(pheno_data) %in% colnames(bvalues),])
if(all_samples == TRUE){
    paste0("All samples in the phenofile found in the beta value matrix")
    bvalues=bvalues
} else {
    missing_samples=rownames(pheno_data[!rownames(pheno_data) %in% colnames(bvalues),])
    pheno_data=pheno_data[!rownames(pheno_data) %in% missing_samples,]
    bvalues=bvalues[, colnames(bvalues) %in% rownames(pheno_data)]
    paste0("The following samples were not found in the beta values but found in the pheno file: ", missing_samples)
}

# ordering pheno file based on beta
paste0("Warning: this script orders the pheno file based on the beta matrix:", Sys.time())
pheno_data=pheno_data[match(rownames(pheno_data),colnames(bvalues)),]

# get available clocks
clocks=availableClock()
# create an empty list for output
out=list()

paste0("Warning, when this script is run for the first time, will install several requiered packages")
paste0("This step could take a while")
# loop over the clocks
for (i in 1:length(clocks)) {
  print(paste0("Processing clock: ",clocks[[i]]))
  mage=methyAge(bvalues, clock=clocks[[i]], age_info=pheno_data, do_plot=TRUE)
  out[[i]]=mage
  names(out[[i]])=c("SampleID",paste0("dnamage_",clocks[[i]],sep=""))
}

out$pheno_data=pheno_data

# saving output
saveRDS(file=outfile,out)